# 4.7 Results for outer-totalistic rules on random 3-regular graphs

In [32]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from src.BP import *
from utils.rules import *

import pickle

plt.rcParams.update({'backend': 'pdf', 'font.size': 14, 'font.family': 'serif', 'text.latex.preamble': r'\usepackage{gensymb}',  'savefig.format':'pdf', 
                     'lines.linewidth': 2, 'lines.markersize':10, 'lines.markeredgewidth':2, 'figure.constrained_layout.w_pad': 2, 'figure.autolayout' : True})

%matplotlib inline

# RS classification on 3-regular graphs

In [4]:
rules=generate_independent_OT_rules(d=3)

In [3]:
fixed_points=[]
phi=[]
for rule in tqdm(rules):
    rule_BP=BP_OT(rule)
    rule_BP.find_all_fixed_points_torch()
    fixed_points.append(rule_BP.fixed_points)
    phi.append(rule_BP.all_phi)
    with open("results/RS_results_3_regular/"+str(rule), "wb") as file:
        pickle.dump(rule_BP, file)

 55%|████████████████████████████████████████████▋                                    | 75/136 [08:02<04:58,  4.90s/it]

Warning, entropy > log(2) even if fixed point physical !
No physical fixed point found for rule ['+', '+', '+', '+'] !


 74%|███████████████████████████████████████████████████████████▍                    | 101/136 [10:50<04:29,  7.70s/it]

No physical fixed point found for rule ['-', '0', '0', '0'] !


 75%|████████████████████████████████████████████████████████████                    | 102/136 [10:54<03:50,  6.78s/it]

...

 86%|████████████████████████████████████████████████████████████████████▊           | 117/136 [12:03<01:28,  4.67s/it]

No physical fixed point found for rule ['-', '-', '0', '-'] !


 88%|██████████████████████████████████████████████████████████████████████          | 119/136 [12:11<01:14,  4.36s/it]

No physical fixed point found for rule ['-', '-', '-', '0'] !


100%|████████████████████████████████████████████████████████████████████████████████| 136/136 [13:18<00:00,  5.87s/it]


In [4]:
with open("results/RS_results_3_regular/fixed_points_list", "wb") as file:
    pickle.dump(fixed_points, file)
with open("results/RS_results_3_regular/phi_list", "wb") as file:
    pickle.dump(phi, file)

In [5]:
with open("results/RS_results_3_regular/fixed_points_list", "rb") as file:
    fixed_points=pickle.load(file)
with open("results/RS_results_3_regular/phi_list", "rb") as file:
    phi=pickle.load(file)

In [8]:
number_homogeneous_configurations=np.zeros(len(rules), dtype=np.int8)
for i, rule in enumerate(rules):
    if rule[0]=='0' or rule[0]=='+':
        number_homogeneous_configurations[i]+=1
    if rule[-1]=='1' or rule[-1]=='+':
        number_homogeneous_configurations[i]+=1

In [6]:
homo_messages=[np.array([[1.,0],[0,0]]), np.array([[0., 0],[0,1]]), np.array([[0.5, 0.5],[0,0]]), np.array([[0,0],[0.5,0.5]])]

In [24]:
epsilon=1e-12
locally_contradictory=[]
no_stationary=[]
only_homo=[]
subexponentially_many_no_homo=[]
subexponentially_many_homo=[]
exponentially_many_no_homogeneous=[]
exponentially_many_with_homogeneous=[]
for i in range(len(rules)):
    BP=None
    with open("results/RS_results_3_regular/"+str(rules[i]), "rb") as file:
        BP=pickle.load(file)
              
    if BP.phi<0-epsilon:
        if BP.phi<-10:
            locally_contradictory.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
        else:
            no_stationary.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
    elif np.abs(BP.phi)<epsilon:
        num_homo=0
        num_phi_0=0
        for j in range(len(BP.all_phi)):
            if abs(BP.all_phi[j])<epsilon:
                num_phi_0+=1
            if np.any(np.all(np.round(BP.fixed_points[j],1) == homo_messages, axis=(1,2))):
                num_homo+=1
        if num_homo!=number_homogeneous_configurations[i]:
            print("Some homogeneous configuration was not found by BP for rule ", rules[i])
        if num_homo==len(BP.all_phi):
            only_homo.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
        else:
            if num_phi_0>num_homo and num_homo>0:
                subexponentially_many_homo.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
            elif num_phi_0>num_homo and num_homo==0:
                subexponentially_many_no_homo.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
            else:
                only_homo.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
                
    elif BP.phi>epsilon and number_homogeneous_configurations[i]==0:
        exponentially_many_no_homogeneous.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
    elif BP.phi>epsilon and number_homogeneous_configurations[i]>0:
        exponentially_many_with_homogeneous.append((rules[i], np.round(BP.phi,4), len(BP.all_phi)))
    else:
        print("problem ....")

In [25]:
data={'Locally contradictory': locally_contradictory, 'No stationary': no_stationary, 'only homogeneous': only_homo, 'Subexponentially many no homogeneous': subexponentially_many_no_homo, 'Subexponentially many with homogeneous':subexponentially_many_homo, 'Exponentially many no homogeneous': exponentially_many_no_homogeneous, 'Exponentially many with homogeneous': exponentially_many_with_homogeneous}

In [26]:
df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.items() ]))

In [19]:
df

,Locally contradictory,No stationary,only homogeneous,Subexponentially many no homogeneous,Subexponentially many with homogeneous,Exponentially many no homogeneous,Exponentially many with homogeneous
0,"([-, 0, 0, 0], -28.2552, 1)",NaN,"([0, 0, 0, 0], 0.0, 1)","([1, 0, 1, 0], 0.0, 4)","([0, 1, 0, 1], 0.0, 3)","([-, 0, +, 0], 0.1016, 2)","([0, 0, +, 0], 0.2046, 2)"
1,"([-, 0, 0, -], -28.2552, 1)",NaN,"([0, 0, 0, +], 0.0, 2)",NaN,"([+, -, +, -], 0.0, 3)","([-, 0, +, -], 0.0523, 2)","([0, 0, +, +], 0.4133, 2)"
2,"([-, 0, -, 0], -28.2552, 1)",NaN,"([0, 0, 0, -], 0.0, 1)",NaN,NaN,"([-, +, 0, 0], 0.2191, 1)","([0, 0, +, -], 0.1925, 2)"
3,"([-, 0, -, -], -28.2552, 1)",NaN,"([0, 0, 0, 1], 0.0, 2)",NaN,NaN,"([-, +, 0, -], 0.1541, 1)","([0, 0, +, 1], 0.3793, 2)"
4,"([-, 0, 1, 0], -0.1116, 3)",NaN,"([0, 0, -, 0], 0.0, 2)",NaN,NaN,"([-, +, +, 0], 0.488, 1)","([0, 0, 1, 0], 0.1204, 4)"
...,...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,NaN,NaN,"([+, 1, 0, +], 0.2964, 3)"
60,NaN,NaN,NaN,NaN,NaN,NaN,"([+, 1, 0, -], 0.1038, 2)"
61,NaN,NaN,NaN,NaN,NaN,NaN,"([+, 1, +, 0], 0.4584, 2)"
62,NaN,NaN,NaN,NaN,NaN,NaN,"([+, 1, +, -], 0.2782, 2)"


In [29]:
print(df[['Locally contradictory','No stationary', 'Subexponentially many no homogeneous']].to_latex(index=False, caption='Classification of outer-totalistic CA in the RS case', longtable=False, na_rep=''))

\begin{table}
\centering
\caption{Classification of outer-totalistic CA in the RS case}
\begin{tabular}{lll}
\toprule
      Locally contradictory &              No stationary & Subexponentially many no homogeneous \\
\midrule
([-, 0, 0, 0], -28.2552, 1) & ([-, 0, 1, 0], -0.1116, 3) &               ([1, 0, 1, 0], 0.0, 4) \\
([-, 0, 0, -], -28.2552, 1) & ([-, 0, 1, -], -0.2027, 3) &                                      \\
([-, 0, -, 0], -28.2552, 1) & ([-, +, -, 0], -0.0774, 2) &                                      \\
([-, 0, -, -], -28.2552, 1) & ([-, +, -, -], -0.1744, 2) &                                      \\
([-, -, 0, 0], -55.0918, 1) & ([-, -, +, 0], -0.0303, 2) &                                      \\
([-, -, 0, -], -55.0918, 1) & ([-, -, 1, 0], -0.1733, 2) &                                      \\
([-, -, -, 0], -45.3644, 1) & ([-, 1, 0, 0], -0.0012, 1) &                                      \\
([-, -, -, -], -34.7619, 1) & ([-, 1, 0, -], -0.2027, 1) &                       

<ipython-input-29-46b696c14773>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[['Locally contradictory','No stationary', 'Subexponentially many no homogeneous']].to_latex(index=False, caption='Classification of outer-totalistic CA in the RS case', longtable=False, na_rep=''))


In [30]:
print(df[['only homogeneous','Subexponentially many with homogeneous','Exponentially many no homogeneous','Exponentially many with homogeneous']].to_latex(index=False, caption='Classification of outer-totalistic CA in the RS case', longtable=False, na_rep=''))

\begin{table}
\centering
\caption{Classification of outer-totalistic CA in the RS case}
\begin{tabular}{llll}
\toprule
       only homogeneous & Subexponentially many with homogeneous & Exponentially many no homogeneous & Exponentially many with homogeneous \\
\midrule
 ([0, 0, 0, 0], 0.0, 1) &                 ([0, 1, 0, 1], 0.0, 3) &         ([-, 0, +, 0], 0.1016, 2) &           ([0, 0, +, 0], 0.2046, 2) \\
 ([0, 0, 0, +], 0.0, 2) &                 ([+, -, +, -], 0.0, 3) &         ([-, 0, +, -], 0.0523, 2) &           ([0, 0, +, +], 0.4133, 2) \\
 ([0, 0, 0, -], 0.0, 1) &                                        &         ([-, +, 0, 0], 0.2191, 1) &           ([0, 0, +, -], 0.1925, 2) \\
 ([0, 0, 0, 1], 0.0, 2) &                                        &         ([-, +, 0, -], 0.1541, 1) &           ([0, 0, +, 1], 0.3793, 2) \\
 ([0, 0, -, 0], 0.0, 2) &                                        &          ([-, +, +, 0], 0.488, 1) &           ([0, 0, 1, 0], 0.1204, 4) \\
 ([0, 0, -, +], 0.0,

<ipython-input-30-83e14fd12030>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[['only homogeneous','Subexponentially many with homogeneous','Exponentially many no homogeneous','Exponentially many with homogeneous']].to_latex(index=False, caption='Classification of outer-totalistic CA in the RS case', longtable=False, na_rep=''))
